In [5]:
import boto
aws_key_id = settings.AWS_ACCESS_KEY_ID
aws_secret_key = settings.AWS_SECRET_ACCESS_KEY
aws_host = settings.AWS_S3_HOST
from boto.s3.connection import S3Connection

In [8]:
conn = S3Connection(
    aws_access_key_id=settings.AWS_ACCESS_KEY_ID,
    aws_secret_access_key=settings.AWS_SECRET_ACCESS_KEY,
    is_secure=False, 
    port=None, 
    proxy=None, 
    proxy_port=None, 
    proxy_user=None, 
    proxy_pass=None, 
    host=settings.AWS_S3_HOST,
    debug=0, 
    https_connection_factory=None, 
    calling_format='boto.s3.connection.SubdomainCallingFormat', 
    path='/', 
    provider='aws', 
    #bucket_class=<class 'boto.s3.bucket.Bucket'>, 
    security_token=None, 
    suppress_consec_slashes=True, 
    anon=False, 
    validate_certs=None, 
    profile_name=None
    )

In [9]:
static = conn.get_bucket(settings.AWS_STORAGE_BUCKET_NAME)

In [71]:
for key in static.get_all_keys(max_keys=3):
    print(key.BufferSize, key.size, key.name, key.etag)

8192 2943229 arkiv/2002/01/01-01.pdf "1c18eb039366f4610af2cfbdb524ace4"
8192 780482 arkiv/2002/01/01-02.pdf "5199237591f774d88cbe4535d926dbe5"
8192 1373175 arkiv/2002/01/01-03.pdf "690c9d3ea81993c7369300ef47e6264c"


In [79]:
import os
key = static.get_key('media/pdf/universitas_2015-28.pdf')

In [80]:
fname = os.path.split(key.name)[1]
key.get_contents_to_filename(fname)

In [108]:
print(key.etag.strip('"'), *file_hashes(fname), sep='\n')
print(key.size, os.stat(fname).st_size) 
print((int(parse_ts(key.last_modified).strftime('%s')) - int(os.stat(fname).st_mtime)) % 60 * 60) 

b054063e5099c3c880bc39520764b354
b054063e5099c3c880bc39520764b354
5992292 5992292
0


In [106]:
key.last_modified

'Wed, 14 Oct 2015 12:48:12 GMT'

In [ ]:
os.stat(fname)

In [103]:

from boto.utils import parse_ts
parse_ts(key.last_modified).strftime('%s')

'1444819692'

In [60]:
filepath=img.file.name.split('no/')[1]
static.get_key(filepath)

<Key: static.universitas.no,media/2007/17/16-Mag-Mote-01-AO.jpg>

In [62]:
print(key)

"b57516e768e202b70641e47cba874c33"


In [65]:
import hashlib
def hashfile(afile, hashalgorithm=hashlib.md5, blocksize=65536):
    buf = afile.read(blocksize)
    hasher = hashalgorithm()
    while len(buf) > 0:
        hasher.update(buf)
        buf = afile.read(blocksize)
    return hasher.hexdigest()

def file_hashes(*filenames):
    hashes = []
    for fname in filenames:
        with open(fname, 'rb') as src:
            hashes.append(hashfile(src))
    return hashes

hashfile(img.file)



'b57516e768e202b70641e47cba874c33'

In [66]:
img.file.size

81763